install libraries

In [1]:
import whisper
import os

from evaluate import load
import audiosegment

d:\ChatterChop\.venv\lib\site-packages\whisper\timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
d:\ChatterChop\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


audio transcription

In [31]:
def transcription(path_to_audio_file, path_to_transcription_file, model_name='small', language='pl'):

    model = whisper.load_model(model_name)
    result = model.transcribe(path_to_audio_file, language=language)

    with open(path_to_transcription_file, "a", encoding="utf-8") as file_txt:
        file_txt.write(f"{result['text']}\n")

test transcription

In [8]:
audio_relative_path = os.path.join("..", "test_data", "test_audio_pl.wav")
transcription_relative_path = os.path.join("..", "test_data", "test_transcription.txt")

In [ ]:
transcription(audio_relative_path, transcription_relative_path)

Transcription metrics

In [6]:
ground_truth = os.path.join("..", "test_data", "test_transcription_ground_truth.txt")
transcription_to_test = os.path.join("..", "test_data", "test_transcription.txt")

In [43]:
def txt_into_list(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            text = text.strip()
            text = ''.join(char.lower() if char.isalnum() or char.isspace() else ' ' for char in text)
            one_long_sentence = ' '.join(text.split())
            sentence_list = [one_long_sentence]
            return sentence_list
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

In [44]:
list_transcription_to_test = txt_into_list(transcription_to_test)
list_ground_truth = txt_into_list(ground_truth)

In [45]:
print(list_transcription_to_test)
print(list_ground_truth)

['warszawa jest miastem pełnym sprzeczności przez wielu nazywanym niezniszczalnym miasto to wzbudza zachwyt prowokuje i inspiruje panującą w nim atmosferę o magicznym duchu nieistniejących już miejsc tworząc unikalne zestawienie zabytkowych budynków z nowoczesną miejską architekturą a przykładem niezwykłej niezłomności stolicy jest jej stare miasto warszawa jest miastem pełnym sprzeczności przez wielu nazywanym niezniszczalnym']
['warszawa jest miastem pełnym sprzeczności przez wielu nazywanym niezniszczalnym miasto to wzbudza zachwyt prowokuje i inspiruje panującą w nim atmosferą o magicznym duchu nieistniejących już miejsc tworząc unikalne zestawienie zabytkowych budynków z nowoczesną miejską architekturą a przykładem niezwykłej niezłomności stolicy jest jej stare miasto warszawa jest miastem pełnym sprzeczności przez wielu nazywanym niezniszczalnym']


In [46]:
wer = load('wer')
wer_score = wer.compute(predictions=list_transcription_to_test, references=list_ground_truth)
print(wer_score)

0.018867924528301886


In [48]:
cer = load("cer")
cer_score = cer.compute(predictions=list_transcription_to_test, references=list_ground_truth)
print(cer_score)

0.002336448598130841


In [66]:
def write_words_to_file(sentence, output_file_path):
    try:
        if len(sentence) == 1:
            words = sentence[0].split()
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                for word in words:
                    output_file.write(word + '\n')
        else:
            print("Invalid input. The 'sentence' parameter should be a list containing one long string.")
    except Exception as e:
        print(f"Error writing to file: {str(e)}")

In [67]:
splitted_words = os.path.join("..", "test_data", "word_by_word.txt")
write_words_to_file(list_ground_truth, splitted_words)

Perform VAD

In [11]:
def vad(audio_path):
    audio = audiosegment.from_file(audio_path)
    speech_segments = []

    # Define the silence audio segment here
    # Make sure it's of type audiosegment
    silence = audiosegment.empty()

    for segment in audio:
        # Check if the segment's dBFS is higher than silence's dBFS
        if segment.dBFS > silence.dBFS:
            current_segment.append(segment)
        else:
            if current_segment:
                speech_segments.append(audiosegment.from_mono_audiosegments(*current_segment))
                current_segment = []

    return speech_segments

Split the Audio

In [4]:
def split_audio(audio_path, transcription):
    if not transcription:
        return

    audio = audiosegment.from_file(audio_path)
    word_segments = []

    for result in transcription.get("alternative", []):
        transcript = result.get("transcript", "")
        if transcript:
            for word_info in result.get("words", []):
                start_time = float(word_info.get("start", 0))
                end_time = float(word_info.get("end", 0))
                word_audio = audio[start_time * 1000:end_time * 1000]
                word_segments.append(word_audio)

    return word_segments

In [12]:
audio_path = os.path.join("..", "test_data", "test_audio_pl.wav")
splitted_words = os.path.join("..", "test_data", "word_by_word.txt")

speech_segments = vad(audio_path)
word_audio_segments = split_audio(audio_path, splitted_words)

# Save the word audio segments as individual files or process as needed
for i, segment in enumerate(word_audio_segments):
    segment.export(f"word_{i}.wav", format="wav")

UnboundLocalError: local variable 'current_segment' referenced before assignment